In [1]:
import os
import sys

PACKAGE_DIR = "/kaggle/src"
sys.path.append(PACKAGE_DIR)
sys.path.append(os.path.join(PACKAGE_DIR, "Penguin-ML-Library"))

In [2]:
import os
import random
import warnings

import numpy as np
import yaml
from penguinml.utils.logger import get_logger, init_logger
from penguinml.utils.set_seed import seed_base

warnings.filterwarnings("ignore")

seed = 46
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)

2024-06-15 06:12:20.605417: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-15 06:12:20.830977: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-15 06:12:21.494467: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/u

In [3]:
import bz2
import gc
import hashlib
import json
import math
import multiprocessing
import random
import time
from glob import glob
from typing import List, Set

import lz4.frame
import matplotlib.pyplot as plt
import numpy as np
import plyvel
import polars as pl
from penguinml.utils.timer import Timer
from tqdm import tqdm

import whoosh_utils
from const import ALL_KEYS, CPC2TOKENS_PATH, INF, KEY2QUERY, NUM_CPU
from solver import SimulatedAnnealing
from utils import (
    calc_bytes,
    compute_ap,
    evaluate,
    load_list_bz2,
    read_bytes_in_range,
    save_list_bz2,
)


def stable_hash(obj, mod: int = 20):
    """
    NOTE: objは`json.dumps`する前のオブジェクト
    """
    obj_str = json.dumps(obj, sort_keys=True)
    hash_bytes = hashlib.sha256(obj_str.encode()).digest()
    hash_int = int.from_bytes(hash_bytes, byteorder="big")
    return hash_int % mod

Processing /kaggle/input/whoosh-wheel-2-7-4/Whoosh-2.7.4-py2.py3-none-any.whl
Whoosh is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [4]:
!rm -rf db
!rm index.lz4

rm: cannot remove 'index.lz4': No such file or directory


In [5]:
files = sorted(glob("/kaggle/input/preprocess-all-token-single/queries*/*.bz2"))
len(files)

13216540

In [6]:
# データベースの作成
dbs = plyvel.DB("db", create_if_missing=True)
txts = open("index.lz4", "wb")


cursors = 0
debug_center2txt = {}
debug_count = 0

pbar = tqdm(files)
for file in pbar:
    exist = set()
    try:
        data = load_list_bz2(file)
    except:
        print(f"{file} is broken")
        continue
    center = os.path.basename(file).split(".")[0]

    # dict -> str
    txt = json.dumps(data)

    start = cursors
    compressed_chunk = lz4.frame.compress(txt.encode("utf-8"))
    txts.write(compressed_chunk)
    txts.flush()
    cursors += len(compressed_chunk)
    end = cursors

    dbs.put(center.encode(), json.dumps((start, end)).encode())

    if debug_count % 10000 == 0:
        debug_center2txt[center] = txt
    debug_count += 1

    # if debug_count > 100000:
    #     break

 59%|█████▉    | 7795572/13216540 [45:31<32:43, 2760.57it/s]  

/kaggle/input/preprocess-all-token-single/queries1/US-2002118634-A1.json.bz2 is broken
/kaggle/input/preprocess-all-token-single/queries1/US-2002118643-A1.json.bz2 is broken


100%|██████████| 13216540/13216540 [1:15:44<00:00, 2908.38it/s] 


In [7]:
def read_lz4_in_range(file_path, start_byte, end_byte):
    with open(file_path, "rb") as f:
        f.seek(start_byte)
        compressed_content = f.read(end_byte - start_byte)
        content = lz4.frame.decompress(compressed_content)
        return content.decode("utf-8")

In [8]:
for center, txt in tqdm(debug_center2txt.items()):
    start, end = json.loads(dbs.get(center.encode()))
    read_txt = read_lz4_in_range("index.lz4", start, end)
    assert txt == read_txt
json.loads(read_txt)

100%|██████████| 1322/1322 [00:00<00:00, 1679.62it/s]


[['detd:hubach', 2, 22, ['US-3752315-A', 'US-3680697-A']],
 ['detd:musschoot', 2, 181, ['US-3680697-A', 'US-3087618-A']],
 ['detd:grizzlies', 2, 213, ['US-4865720-A', 'US-3087618-A']],
 ['clm:heavies', 2, 250, ['US-2012199519-A1', 'US-6446813-B1']],
 ['detd:otoole', 2, 272, ['US-2009273-A', 'US-1951705-A']],
 ['detd:trommels', 2, 292, ['US-2022288640-A1', 'US-4867866-A']]]

In [10]:
dbs.close()
txts.close()

In [11]:
!zip -r db.zip db > /dev/null